# Selecting Variants Associated with Bone Fracture

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score, mutual_info_score, confusion_matrix, f1_score
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from matplotlib import figure
import matplotlib.cm as cm
import pickle
import random
import pandas as pd
from tqdm.notebook import tqdm


sns.set()


Bad key "text.kerning_factor" on line 4 in
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Loading Binarized SNPs and applying simple ML Algorithms

In [2]:
rawdata = pd.read_csv('chr3_10000_30000.csv', index_col=False)
rawdata = rawdata.iloc[:,1:]
rawdata.insert(0, 'label', ['frac','frac','frac','frac','normal','normal','normal','normal','normal','normal','normal','normal'])

y = rawdata.iloc[:,0]
X = rawdata.iloc[:,1:-1]
print(X)
print(y)
svmclf = svm.SVC(kernel='linear', C=1)
svmScores = cross_val_score(svmclf, X, y, cv=2)
rfclf = RandomForestClassifier(n_estimators=15, 
                               bootstrap = True,
                               max_features = 'sqrt')
rfScores = cross_val_score(rfclf, X, y, cv=2)
logclf = LogisticRegression(random_state=0, class_weight = 'balanced')
logScores = cross_val_score(logclf, X, y, cv=6)
print(svmScores, rfScores, logScores)
print("Accuracy svm: %0.2f (+/- %0.2f)" % (svmScores.mean(), svmScores.std() * 2))
print("Accuracy rf: %0.2f (+/- %0.2f)" % (rfScores.mean(), rfScores.std() * 2))

svmclf.fit(X,y)
logclf.fit(X,y)
rfclf.fit(X,y)

print(svmclf.predict(X.iloc[0:12,:]))
print(logclf.predict(X.iloc[0:12,:]))
print(rfclf.predict(X.iloc[0:12,:]))
# loo = LeaveOneOut()
# loo.get_n_splits(X)
# print(loo)
# LeaveOneOut()
# for train_index, test_index in loo.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     print(X_train, X_test, y_train, y_test)

# logistic regression scikit-learn class balance. 
# Modify class_weight = balanced

    361492  361659  367621  367756  367824  370050  384917  386246  386519  \
0        0       0       1       0       0       0       0       1       1   
1        0       0       0       0       0       0       1       0       1   
2        0       0       0       0       0       1       1       1       0   
3        0       0       0       0       0       0       1       1       0   
4        1       0       0       0       0       0       0       1       0   
5        0       0       0       0       0       0       0       0       1   
6        0       0       0       0       0       0       0       0       1   
7        0       0       0       0       0       0       0       0       1   
8        0       0       0       0       0       0       1       0       1   
9        0       1       0       0       0       0       1       0       1   
10       0       0       0       0       0       0       1       1       0   
11       0       0       0       1       1       1       1      

/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=6.
  % (min_groups, self.n_splits)), Warning)
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence thi

[0.66666667 0.66666667] [0.5        0.66666667] [0.66666667 0.33333333 0.5        0.5        1.         0.        ]
Accuracy svm: 0.67 (+/- 0.00)
Accuracy rf: 0.58 (+/- 0.17)
['frac' 'frac' 'frac' 'frac' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal']
['frac' 'frac' 'frac' 'frac' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal']
['frac' 'frac' 'frac' 'frac' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal']


/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## **Selecting Variants based on Mutual Information Score**
1. Rank and sort SNP loci based on Mutual Information with each class label. E.g [1955 1251  689 ..., 1281  205    0]
2. Select and add the SNP locus from high to low MI value only if the correlation value is < 0.5 with all the previously selected SNPs.
3. Train the model with SVM (Training Set)
4. Compare the performance difference using All SNPs vs Selected SNPs (Test Set)​

In [3]:
def modifiedFS(data, label, X_train, X_test, y_train, y_test, train_index, test_index, num_features, threshold):
    sortedMIList = []
    labelDict = {} # Stores the indices of high MI features according to labels
    # print(data.shape)
    sample_index = [i for i in range(data.shape[0])]
    popList = ['normal', 'frac']
    for pop in popList:
        mutualInfos = []
        pop_label = y_train
        #print(pop_label)
        pop_label.loc[pop_label.iloc[:] != pop] = 0
        pop_label.loc[pop_label.iloc[:] == pop] = 1
        #label[1] = temp
        print(pop_label.sum())
        for i in range(X_train.shape[1]):
            mutualInfos.append(mutual_info_score(pop_label, X_train.iloc[:,i]))
        sortedMI = np.array(mutualInfos).argsort()
        sortedMI = sortedMI[::-1]
        print('This is the sorted MI index for {} :'.format(pop) , sortedMI)
        sortedMIList.append(sortedMI)
        labelDict[pop] = sortedMI
    print(labelDict)
    featureIndexes = []
    sortedMI = []
    features = pd.DataFrame()
    MI_indices_pop = pd.DataFrame.from_dict(labelDict)
    [row, column] = MI_indices_pop.shape
    for i in range(row) :
        for j in range(column):
            sortedMI.append(MI_indices_pop.iloc[i, j])
    print(len(sortedMI))
    #print(sortedMI)
    features = pd.concat([data.iloc[:,sortedMI[0]], data.iloc[:,sortedMI[2]]], axis=1)
    featureIndexes = [sortedMI[0], sortedMI[2]]
    accuracyList = []
    idx = 2
    while len(featureIndexes) <= num_features:
        # print('The correlation value for index {},{}: '.format(idx, idx+1), features.iloc[:,-1].corr(data.iloc[:,sortedMI[idx]]))
        while sortedMI[idx] in featureIndexes:
            idx = idx+1
        print('Num iteration : ', idx)
        cor = False
        for i in range(features.shape[1]):
            if abs(features.iloc[:,i].corr(data.iloc[:,sortedMI[idx]])) > threshold:
                cor = True
                print(cor)
                break
        if cor:
            idx = idx +1
            continue
            
#         if abs(features.iloc[:,-1].corr(data.iloc[:,sortedMI[idx]])) > threshold:
#             idx = idx +1
#             continue
            
        else :
            #reducedX = pd.concat([label, pd.concat([features, data.iloc[:,sortedMI[idx]]], axis=1)], axis=1)
            features = pd.concat([features, data.iloc[:,sortedMI[idx]]], axis =1)
            featureIndexes.append(sortedMI[idx])
            idx = idx +1
            # print(featureIndexes)
            # print("This is index of features: ", featureIndexes)
#         if len(featureIndexes) %13 ==0:
#             os.remove("data/*.nex")
#             print("network files removed")
#         print(featureIndexes)
    return data.iloc[:,featureIndexes]
    

## Validating the results by Leave One Out Cross Validation

In [4]:
def getAccu(X, clf, numFeat, threshold):
    loo = LeaveOneOut()
    accu = 0
    f1 = 0
    for train, test in loo.split(X):
        print("%s %s" % (train, test))
        print(X.iloc[train,:])
        newX = modifiedFS(X, y, X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test], train, test, numFeat, threshold)
        clf.fit(newX.iloc[train], y.iloc[train])
#         print(clf.score(newX.iloc[test], y.iloc[test]))
#         print(f1_score(clf.predict(newX.iloc[test]), y.iloc[test], average='weighted'))
        accu += clf.score(newX.iloc[test], y.iloc[test])
        f1 += f1_score(clf.predict(newX.iloc[test]), y.iloc[test], average='weighted')
    return accu/X.shape[0], f1/X.shape[0]

In [ ]:
def testAcc(clf):
    interval = 2
    accuList = []
    f1List = []
    for i in tqdm(range(1,21)):
        ac, f1 = getAccu(X, clf, interval*i, 0.8)
        accuList.append(ac)
        f1List.append(f1)
    return accuList, f1List
    
svmAccu, svmf1 = testAcc(svmclf)
ranAccu, ranf1 = testAcc(rfclf)
# logAccu, logf1 = testAcc(logclf)

[ 1  2  3  4  5  6  7  8  9 10 11] [0]
    361492  361659  367621  367756  367824  370050  384917  386246  386519  \
1        0       0       0       0       0       0       1       0       1   
2        0       0       0       0       0       1       1       1       0   
3        0       0       0       0       0       0       1       1       0   
4        1       0       0       0       0       0       0       1       0   
5        0       0       0       0       0       0       0       0       1   
6        0       0       0       0       0       0       0       0       1   
7        0       0       0       0       0       0       0       0       1   
8        0       0       0       0       0       0       1       0       1   
9        0       1       0       0       0       0       1       0       1   
10       0       0       0       0       0       0       1       1       0   
11       0       0       0       1       1       1       1       1       0   

    390915  ...  1978476

/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


This is the sorted MI index for frac : [ 8126  5464  9282 ...  8360 11092  6215]
{'normal': array([ 5685,  4657,   497, ..., 12452, 12447,  1092]), 'frac': array([ 8126,  5464,  9282, ...,  8360, 11092,  6215])}
25740
Num iteration :  3
[ 0  2  3  4  5  6  7  8  9 10 11] [1]
    361492  361659  367621  367756  367824  370050  384917  386246  386519  \
0        0       0       1       0       0       0       0       1       1   
2        0       0       0       0       0       1       1       1       0   
3        0       0       0       0       0       0       1       1       0   
4        1       0       0       0       0       0       0       1       0   
5        0       0       0       0       0       0       0       0       1   
6        0       0       0       0       0       0       0       0       1   
7        0       0       0       0       0       0       0       0       1   
8        0       0       0       0       0       0       1       0       1   
9        0       1    

/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


This is the sorted MI index for normal : [2858 5551 5767 ... 8293 2512 3300]
0


...

/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


This is the sorted MI index for normal : [ 7908 10815  1081 ...  8005  2290  9017]
0


/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


This is the sorted MI index for frac : [9498 7180 2413 ... 4458 2277 6125]
{'normal': array([ 7908, 10815,  1081, ...,  8005,  2290,  9017]), 'frac': array([9498, 7180, 2413, ..., 4458, 2277, 6125])}
25740
Num iteration :  3
Num iteration :  4
True
Num iteration :  5
Num iteration :  6
True
Num iteration :  7
Num iteration :  8
True
Num iteration :  9
Num iteration :  10
True
Num iteration :  11
Num iteration :  12
True
Num iteration :  13
Num iteration :  14
True
Num iteration :  15
Num iteration :  16
True
Num iteration :  17
Num iteration :  18
True
Num iteration :  19
Num iteration :  20
True
Num iteration :  21
True
Num iteration :  22
True
Num iteration :  23
True
Num iteration :  24
Num iteration :  25
Num iteration :  26
Num iteration :  27
Num iteration :  28
True
Num iteration :  29
Num iteration :  30
True
Num iteration :  31
Num iteration :  32
True
Num iteration :  33
True
Num iteration :  34
True
Num iteration :  35
Num iteration :  36
Num iteration :  37
Num iteration : 

/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Draw the Plot

In [ ]:
plt.figure()
plt.plot([i*interval for i in range(1,11)], accuList[:10], 'yellow', label='Accuracy')
plt.plot([i*interval for i in range(1,11)], ranAccu, 'teal', label='Accuracy')
# plt.plot([i*interval for i in range(1,21)], f1List, 'cyan', label='f1 score')
# plt.legend(bbox_to_anchor=(1.0, 1), loc=2, borderaxespad=0.)
plt.hlines(0.66, 0, 50, colors='k', linestyles='solid', label='Accuracy with all SNPs')
plt.xlabel('Number of SNPs selected')
plt.ylabel('Performance')
plt.title('Accuracy and Accuracy with different number of SNPs selected')
plt.show(block = False)
#kalisto - for gene expression

## Ignore below, it is for testing different stuffs

In [ ]:
def getIndices(data, label, num_features, threshold):
    sortedMIList = []
    labelDict = {} # Stores the indices of high MI features according to labels
    # print(data.shape)
    sample_index = [i for i in range(data.shape[0])]
    popList = ['normal', 'frac']
    for pop in popList:
        mutualInfos = []
        pop_label = label
        #print(pop_label)
        pop_label.loc[pop_label.iloc[:] != pop] = 0
        pop_label.loc[pop_label.iloc[:] == pop] = 1
        #label[1] = temp      print(len(featureIndexes), num_features)
        print(pop_label.sum())
        for i in range(data.shape[1]):
            mutualInfos.append(mutual_info_score(pop_label, data.iloc[:,i]))
        sortedMI = np.array(mutualInfos).argsort()
        sortedMI = sortedMI[::-1]
        print('This is the sorted MI index for {} :'.format(pop) , sortedMI)
        sortedMIList.append(sortedMI)
        labelDict[pop] = sortedMI
    print(labelDict)
    featureIndexes = []
    sortedMI = []
    features = pd.DataFrame()
    MI_indices_pop = pd.DataFrame.from_dict(labelDict)
    [row, column] = MI_indices_pop.shape
    for i in range(row) :
        for j in range(column):
            sortedMI.append(MI_indices_pop.iloc[i, j])
    print(len(sortedMI))
    #print(sortedMI)
    features = pd.concat([data.iloc[:,sortedMI[0]], data.iloc[:,sortedMI[1]]], axis=1)
    featureIndexes = [sortedMI[0], sortedMI[2]]
    print(featureIndexes)
    accuracyList = []
    idx = 2
    while len(featureIndexes) < num_features:
        while sortedMI[idx] in featureIndexes:
            idx = idx+1
        print('Num iteration : ', idx)
        cor = False
        for i in range(features.shape[1]):
            if abs(features.iloc[:,i].corr(data.iloc[:,sortedMI[idx]])) > threshold:
                cor = True
                print(cor)
                break
        if cor:
            idx = idx +1
            continue
            
        else :
            features = pd.concat([features, data.iloc[:,sortedMI[idx]]], axis =1)
            featureIndexes.append(sortedMI[idx])
            idx = idx +1
            
    return featureIndexes



In [ ]:
indices = getIndices(X, y, 50, 1.0)
print(indices)

In [ ]:
print(X.iloc[:,indices].columns)
print(list(X.iloc[:,indices].columns))

In [ ]:
for item in list(X.iloc[:,indices].columns):
    print(X[item])
    
    
for i in range(33155572, 33171517):
    try:
        print(X[str(i)])
    except KeyError:
        continue

In [ ]:
for item in list(X.iloc[:,indices].columns):
    print(mutual_info_score(y, X[item]))